In [96]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime, timedelta
import yfinance as yf
from scipy.stats import mannwhitneyu


In [114]:
#Total movement 8/23 - 9/13
def get_stock_movement(ticker, start_date, end_date):
    try:
        # Adjust the end_date to include the intended day
        end_date_adjusted = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

        stock_data = yf.download(ticker, start=start_date, end=end_date_adjusted, progress=False)

        if stock_data.empty:
            return {"error": f"No data found for {ticker} between {start_date} and {end_date}"}

        opening_price = stock_data['Open'].iloc[0]
        closing_price = stock_data['Close'].iloc[-1]
        peak_price = stock_data['High'].max()

        percentage_total = ((closing_price - opening_price) / opening_price) * 100

        return {
            "ticker": ticker,
            "8/26-9/16_total": round(percentage_total, 2)  # Rounded to two decimal places
        }

    except Exception as e:
        return {"error": str(e)}

# List of tickers
tickers = ['NVA.AX', 'CCTG', 'RETO', 'DPRO', 'UAVS', 'GDC', 'KNW', 'MVO', 'PRE', 'GLLI', 'WPRT', 'RR', 'LEV',
           'BNGO', 'NISN', 'HUDI', 'DTCK', 'ZKIN', 'PMEC', 'HYFM', 'TPCS', 'FEAM', 'AXDX', 'VGAS', 'MTEN',
            'DSWL', 'GLST', 'MIND', 'FSI', 'IPWR', 'ESGL', 'CPTN', 'CLIR', 'JUNE', 'APT', 'TPIC', 'GWH', 'PPSI',
            'NWGL', 'PZG', 'WWR', 'XPL', 'HYMC', 'NB', 'FURY', 'VGZ', 'USAU', 'AMLI', 'AUST', 'GORO', 'TRX',
            'THM', 'USEG', 'VIVK', 'PRT', 'DWSN', 'BATL', 'RCON', 'CRT', 'PED', 'NCSM', 'KLXE', 'AREC', 'AE',
            ]
           

# Start and end dates
start_date = "2024-08-26"
end_date = "2024-09-16"

# Initialize empty list to hold the data
data = []

# Loop through tickers
for ticker in tickers:
    result = get_stock_movement(ticker, start_date, end_date)
    if "error" in result:
        print(f"Error for {ticker}: {result['error']}")
    else:
        data.append(result)

# Create DataFrame
nva_comp_df = pd.DataFrame(data)

# Display the DataFrame with the specified columns
nva_comp_df = nva_comp_df[['ticker', '8/26-9/16_total']].astype({'ticker': str, '8/26-9/16_total': float})

print(nva_comp_df)


    ticker  8/26-9/16_total
0   NVA.AX            -3.23
1     CCTG             3.76
2     RETO            -7.79
3     DPRO           -20.99
4     UAVS           -40.00
..     ...              ...
59     PED            11.55
60    NCSM             0.14
61    KLXE           -33.60
62    AREC            -5.66
63      AE             2.76

[64 rows x 2 columns]


C:\Users\19413\AppData\Local\Temp\ipykernel_16660\1908368385.py:54: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nva_comp_df = nva_comp_df[['ticker', '8/26-9/16_total']].astype({'ticker': str, '8/26-9/16_total': float})


In [116]:
just_nva_total = nva_comp_df.loc[nva_comp_df['ticker'] == 'NVA.AX', '8/26-9/16_total'].values[0]
rest_totals = nva_comp_df.loc[nva_comp_df['ticker'] != 'NVA.AX', '8/26-9/16_total']

import numpy as np
from scipy import stats
##Perplexity version
def hodges_lehmann_interval(just_nva_total, rest_totals, alpha=0.05):
    if np.isscalar(just_nva_total):
        just_nva_total = [just_nva_total]
    if np.isscalar(rest_totals):
        rest_totals = [rest_totals]

    pairwise_differences = [x1 - x2 for x1 in just_nva_total for x2 in rest_totals]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n1, n2 = len(just_nva_total), len(rest_totals)
    n = n1 * n2

    # Calculate the critical value
    z = stats.norm.ppf(1 - alpha/2)

    # Calculate the standard error
    se = np.sqrt((n1 + n2 + 1) / (12 * n1 * n2))

    # Calculate the confidence interval
    margin = z * se * np.sqrt(n)
    lower_rank = int(np.ceil((n - margin) / 2) - 1)
    upper_rank = int(np.floor((n + margin) / 2) - 1)

    lower_bound = pairwise_differences[max(0, lower_rank)]
    upper_bound = pairwise_differences[min(n - 1, upper_rank)]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

# The rest of your code remains the same
u_stat, p_value = stats.mannwhitneyu(just_nva_total, rest_totals, alternative='two-sided')

result = hodges_lehmann_interval(just_nva_total, rest_totals, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")



Mann Whitney U Statistics: 32.0
p value: 1.0000
Hodges-Lehmann Estimator: 0.43000000000000016
Confidence Interval: (np.float64(-0.79), np.float64(0.52))


In [119]:
# Peak movement 8/22 - 9/16
def get_stock_movement(ticker, start_date, end_date):
    try:
        # Adjust the end_date to include the intended day
        end_date_adjusted = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

        stock_data = yf.download(ticker, start=start_date, end=end_date_adjusted, progress=False)

        if stock_data.empty:
            return {"error": f"No data found for {ticker} between {start_date} and {end_date}"}

        opening_price = stock_data['Open'].iloc[0]
        peak_price = stock_data['High'].max()

        percentage_peak = ((peak_price - opening_price) / opening_price) * 100

        return {
            "ticker": ticker,
            "8/22-9/16_peak": round(percentage_peak, 2)  # Rounded to two decimal places
        }

    except Exception as e:
        return {"error": str(e)}

# List of tickers
tickers = ['NVA.AX', 'CCTG', 'RETO', 'DPRO', 'UAVS', 'GDC', 'KNW', 'MVO', 'PRE', 'GLLI', 'WPRT', 'RR', 'LEV',
           'BNGO', 'NISN', 'HUDI', 'DTCK', 'ZKIN', 'PMEC', 'HYFM', 'TPCS', 'FEAM', 'AXDX', 'VGAS', 'MTEN',
            'DSWL', 'GLST', 'MIND', 'FSI', 'IPWR', 'ESGL', 'CPTN', 'CLIR', 'JUNE', 'APT', 'TPIC', 'GWH', 'PPSI',
            'NWGL', 'PZG', 'WWR', 'XPL', 'HYMC', 'NB', 'FURY', 'VGZ', 'USAU', 'AMLI', 'AUST', 'GORO', 'TRX',
            'THM', 'USEG', 'VIVK', 'PRT', 'DWSN', 'BATL', 'RCON', 'CRT', 'PED', 'NCSM', 'KLXE', 'AREC', 'AE',
            ]

start_date = "2024-08-22"
end_date = "2024-09-17"

data = []

for ticker in tickers:
    result = get_stock_movement(ticker, start_date, end_date)
    if "error" in result:
        print(f"Error for {ticker}: {result['error']}")
    else:
        data.append(result)

nva_comp_df = pd.DataFrame(data)

# Display the DataFrame with the specified columns
nva_comp_df = nva_comp_df[['ticker', '8/22-9/16_peak']].astype({'ticker': str, '8/22-9/16_peak': float})

print(nva_comp_df)


    ticker  8/22-9/16_peak
0   NVA.AX           18.75
1     CCTG           15.98
2     RETO           80.20
3     DPRO            0.71
4     UAVS           30.00
..     ...             ...
59     PED           18.99
60    NCSM            6.33
61    KLXE            7.13
62    AREC            7.46
63      AE           12.52

[64 rows x 2 columns]


C:\Users\19413\AppData\Local\Temp\ipykernel_16660\661485946.py:48: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nva_comp_df = nva_comp_df[['ticker', '8/22-9/16_peak']].astype({'ticker': str, '8/22-9/16_peak': float})


In [120]:
just_nva_peak = nva_comp_df.loc[nva_comp_df['ticker'] == 'NVA.AX', '8/22-9/16_peak'].values[0]
rest_peaks = nva_comp_df.loc[nva_comp_df['ticker'] != 'NVA.AX', '8/22-9/16_peak']

import numpy as np
from scipy import stats
##Perplexity version
def hodges_lehmann_interval(just_nva_peak, rest_peaks, alpha=0.05):
    if np.isscalar(just_nva_peak):
        just_nva_peak = [just_nva_peak]
    if np.isscalar(rest_peaks):
        rest_peaks = [rest_peaks]

    pairwise_differences = [x1 - x2 for x1 in just_nva_peak for x2 in rest_peaks]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n1, n2 = len(just_nva_peak), len(rest_peaks)
    n = n1 * n2

    # Calculate the critical value
    z = stats.norm.ppf(1 - alpha/2)

    # Calculate the standard error
    se = np.sqrt((n1 + n2 + 1) / (12 * n1 * n2))

    # Calculate the confidence interval
    margin = z * se * np.sqrt(n)
    lower_rank = int(np.ceil((n - margin) / 2) - 1)
    upper_rank = int(np.floor((n + margin) / 2) - 1)

    lower_bound = pairwise_differences[max(0, lower_rank)]
    upper_bound = pairwise_differences[min(n - 1, upper_rank)]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

# The rest of your code remains the same
u_stat, p_value = stats.mannwhitneyu(just_nva_peak, rest_peaks, alternative='two-sided')

result = hodges_lehmann_interval(just_nva_peak, rest_peaks, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")



Mann Whitney U Statistics: 46.0
p value: 0.4485
Hodges-Lehmann Estimator: 11.530000000000001
Confidence Interval: (np.float64(11.25), np.float64(11.620000000000001))


In [117]:
#Total movement 9/30 - 10/24
def get_stock_movement(ticker, start_date, end_date):
    try:
        # Adjust the end_date to include the intended day
        end_date_adjusted = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

        stock_data = yf.download(ticker, start=start_date, end=end_date_adjusted, progress=False)

        if stock_data.empty:
            return {"error": f"No data found for {ticker} between {start_date} and {end_date}"}

        opening_price = stock_data['Open'].iloc[0]
        closing_price = stock_data['Close'].iloc[-1]
        peak_price = stock_data['High'].max()

        percentage_total = ((closing_price - opening_price) / opening_price) * 100

        return {
            "ticker": ticker,
            "9/30-10/24_total": round(percentage_total, 2)  # Rounded to two decimal places
        }

    except Exception as e:
        return {"error": str(e)}

# List of tickers
tickers = ['NVA.AX', 'CCTG', 'RETO', 'DPRO', 'UAVS', 'GDC', 'KNW', 'MVO', 'PRE', 'GLLI', 'WPRT', 'RR', 'LEV',
           'BNGO', 'NISN', 'HUDI', 'DTCK', 'ZKIN', 'PMEC', 'HYFM', 'TPCS', 'FEAM', 'AXDX', 'VGAS', 'MTEN',
            'DSWL', 'GLST', 'MIND', 'FSI', 'IPWR', 'ESGL', 'CPTN', 'CLIR', 'JUNE', 'APT', 'TPIC', 'GWH', 'PPSI',
            'NWGL', 'PZG', 'WWR', 'XPL', 'HYMC', 'NB', 'FURY', 'VGZ', 'USAU', 'AMLI', 'AUST', 'GORO', 'TRX',
            'THM', 'USEG', 'VIVK', 'PRT', 'DWSN', 'BATL', 'RCON', 'CRT', 'PED', 'NCSM', 'KLXE', 'AREC', 'AE',
            ]
           

# Start and end dates
start_date = "2024-09-30"
end_date = "2024-10-24"

# Initialize empty list to hold the data
data = []

# Loop through tickers
for ticker in tickers:
    result = get_stock_movement(ticker, start_date, end_date)
    if "error" in result:
        print(f"Error for {ticker}: {result['error']}")
    else:
        data.append(result)

# Create DataFrame
nva_comp_df = pd.DataFrame(data)

# Display the DataFrame with the specified columns
nva_comp_df = nva_comp_df[['ticker', '9/30-10/24_total']].astype({'ticker': str, '9/30-10/24_total': float})

print(nva_comp_df)


    ticker  9/30-10/24_total
0   NVA.AX             92.59
1     CCTG             41.76
2     RETO            -14.84
3     DPRO             10.66
4     UAVS            -87.26
..     ...               ...
59     PED             -0.54
60    NCSM            -11.65
61    KLXE            -18.85
62    AREC             12.46
63      AE              5.65

[64 rows x 2 columns]


C:\Users\19413\AppData\Local\Temp\ipykernel_16660\349169284.py:54: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nva_comp_df = nva_comp_df[['ticker', '9/30-10/24_total']].astype({'ticker': str, '9/30-10/24_total': float})


In [118]:
#For peak movement 9/30 - 10/24

just_nva_total = nva_comp_df.loc[nva_comp_df['ticker'] == 'NVA.AX', '9/30-10/24_total'].values[0]
rest_total = nva_comp_df['9/30-10/24_total'][1:]

import numpy as np
from scipy import stats
##Perplexity version
def hodges_lehmann_interval(just_nva_total, rest_total, alpha=0.05):
    if np.isscalar(just_nva_total):
        just_nva_total = [just_nva_total]
    if np.isscalar(rest_total):
        rest_total = [rest_total]

    pairwise_differences = [x1 - x2 for x1 in just_nva_total for x2 in rest_total]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n1, n2 = len(just_nva_total), len(rest_total)
    n = n1 * n2

    # Calculate the critical value
    z = stats.norm.ppf(1 - alpha/2)

    # Calculate the standard error
    se = np.sqrt((n1 + n2 + 1) / (12 * n1 * n2))

    # Calculate the confidence interval
    margin = z * se * np.sqrt(n)
    lower_rank = int(np.ceil((n - margin) / 2) - 1)
    upper_rank = int(np.floor((n + margin) / 2) - 1)

    lower_bound = pairwise_differences[max(0, lower_rank)]
    upper_bound = pairwise_differences[min(n - 1, upper_rank)]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

# The rest of your code remains the same
u_stat, p_value = stats.mannwhitneyu(just_nva_total, rest_total, alternative='two-sided')

result = hodges_lehmann_interval(just_nva_total, rest_total, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")


Mann Whitney U Statistics: 62.0
p value: 0.1044
Hodges-Lehmann Estimator: 91.4
Confidence Interval: (np.float64(90.66), np.float64(91.4))


In [102]:
just_nva_total = nva_comp_df.loc[nva_comp_df['ticker'] == 'NVA.AX', '9/30-10/24_total'].values[0]
rest_totals = nva_comp_df.loc[nva_comp_df['ticker'] != 'NVA.AX', '9/30-10/24_total']


def hodges_lehmann_interval(just_nva_total, rest_totals, alpha=0.05):
    if np.isscalar(just_nva_total):
        just_nva_total = [just_nva_total]
    if np.isscalar(rest_totals):
        rest_totals = [rest_totals]

    pairwise_differences = [x1 - x2 for x1 in just_nva_total for x2 in rest_totals]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n = len(pairwise_differences)

    _, u_stat = mannwhitneyu(just_nva_total, rest_totals, alternative='two-sided')

    lower_rank = int(max(0, u_stat - 1.96 * np.sqrt(n)))  # Ensure lower rank is >= 0
    upper_rank = int(min(n - 1, u_stat + 1.96 * np.sqrt(n)))  # Ensure upper rank < n

    lower_bound = pairwise_differences[lower_rank]
    upper_bound = pairwise_differences[upper_rank]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

u_stat, p_value = stats.mannwhitneyu(just_nva_total, rest_totals, alternative = 'two-sided')


result = hodges_lehmann_interval(just_nva_total, rest_totals, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")


Mann Whitney U Statistics: 62.0
p value: 0.1044
Hodges-Lehmann Estimator: 91.4
Confidence Interval: (np.float64(-10.920000000000002), np.float64(84.26))


In [ ]:
import numpy as np
from scipy import stats
##Perplexity version
def hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05):
    if np.isscalar(just_gp_peak):
        just_gp_peak = [just_gp_peak]
    if np.isscalar(rest_peak):
        rest_peak = [rest_peak]

    pairwise_differences = [x1 - x2 for x1 in just_gp_peak for x2 in rest_peak]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n1, n2 = len(just_gp_peak), len(rest_peak)
    n = n1 * n2

    # Calculate the critical value
    z = stats.norm.ppf(1 - alpha/2)

    # Calculate the standard error
    se = np.sqrt((n1 + n2 + 1) / (12 * n1 * n2))

    # Calculate the confidence interval
    margin = z * se * np.sqrt(n)
    lower_rank = int(np.ceil((n - margin) / 2) - 1)
    upper_rank = int(np.floor((n + margin) / 2) - 1)

    lower_bound = pairwise_differences[max(0, lower_rank)]
    upper_bound = pairwise_differences[min(n - 1, upper_rank)]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

# The rest of your code remains the same
u_stat, p_value = stats.mannwhitneyu(just_gp_peak, rest_peak, alternative='two-sided')

result = hodges_lehmann_interval(just_gp_peak, rest_peak, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")


In [121]:
#Peak movement 9/30 - 10/24
def get_stock_movement(ticker, start_date, end_date):
    try:
        # Adjust the end_date to include the intended day
        end_date_adjusted = (datetime.strptime(end_date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")

        stock_data = yf.download(ticker, start=start_date, end=end_date_adjusted, progress=False)

        if stock_data.empty:
            return {"error": f"No data found for {ticker} between {start_date} and {end_date}"}

        opening_price = stock_data['Open'].iloc[0]
        peak_price = stock_data['High'].max()

        percentage_peak = ((peak_price - opening_price) / opening_price) * 100

        return {
            "ticker": ticker,
            "9/30-10/24_peak": round(percentage_peak, 2)  # Rounded to two decimal places
        }

    except Exception as e:
        return {"error": str(e)}

# List of tickers
tickers = ['NVA.AX', 'CCTG', 'RETO', 'DPRO', 'UAVS', 'GDC', 'KNW', 'MVO', 'PRE', 'GLLI', 'WPRT', 'RR', 'LEV',
           'BNGO', 'NISN', 'HUDI', 'DTCK', 'ZKIN', 'PMEC', 'HYFM', 'TPCS', 'FEAM', 'AXDX', 'VGAS', 'MTEN',
            'DSWL', 'GLST', 'MIND', 'FSI', 'IPWR', 'ESGL', 'CPTN', 'CLIR', 'JUNE', 'APT', 'TPIC', 'GWH', 'PPSI',
            'NWGL', 'PZG', 'WWR', 'XPL', 'HYMC', 'NB', 'FURY', 'VGZ', 'USAU', 'AMLI', 'AUST', 'GORO', 'TRX',
            'THM', 'USEG', 'VIVK', 'PRT', 'DWSN', 'BATL', 'RCON', 'CRT', 'PED', 'NCSM', 'KLXE', 'AREC', 'AE',
            ]

# Start and end dates
start_date = "2024-09-30"
end_date = "2024-10-24"

# Initialize empty list to hold the data
data = []

# Loop through tickers
for ticker in tickers:
    result = get_stock_movement(ticker, start_date, end_date)
    if "error" in result:
        print(f"Error for {ticker}: {result['error']}")
    else:
        data.append(result)

# Create DataFrame
nva_comp_df = pd.DataFrame(data)

# Display the DataFrame with the specified columns
nva_comp_df = nva_comp_df[['ticker', '9/30-10/24_peak']].astype({'ticker': str, '9/30-10/24_peak': float})

print(nva_comp_df)


    ticker  9/30-10/24_peak
0   NVA.AX           103.70
1     CCTG            86.47
2     RETO            21.09
3     DPRO            63.52
4     UAVS             0.00
..     ...              ...
59     PED            12.15
60    NCSM             0.00
61    KLXE             9.24
62    AREC            31.43
63      AE            12.09

[64 rows x 2 columns]


C:\Users\19413\AppData\Local\Temp\ipykernel_16660\692376834.py:52: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nva_comp_df = nva_comp_df[['ticker', '9/30-10/24_peak']].astype({'ticker': str, '9/30-10/24_peak': float})


In [122]:
#For peak movement 9/30 - 10/24

just_nva_peak = nva_comp_df.loc[nva_comp_df['ticker'] == 'NVA.AX', '9/30-10/24_peak'].values[0]
rest_peak = nva_comp_df['9/30-10/24_peak'][1:]

import numpy as np
from scipy import stats
##Perplexity version
def hodges_lehmann_interval(just_nva_peak, rest_peak, alpha=0.05):
    if np.isscalar(just_nva_peak):
        just_nva_peak = [just_nva_peak]
    if np.isscalar(rest_peak):
        rest_peak = [rest_peak]

    pairwise_differences = [x1 - x2 for x1 in just_nva_peak for x2 in rest_peak]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n1, n2 = len(just_nva_peak), len(rest_peak)
    n = n1 * n2

    # Calculate the critical value
    z = stats.norm.ppf(1 - alpha/2)

    # Calculate the standard error
    se = np.sqrt((n1 + n2 + 1) / (12 * n1 * n2))

    # Calculate the confidence interval
    margin = z * se * np.sqrt(n)
    lower_rank = int(np.ceil((n - margin) / 2) - 1)
    upper_rank = int(np.floor((n + margin) / 2) - 1)

    lower_bound = pairwise_differences[max(0, lower_rank)]
    upper_bound = pairwise_differences[min(n - 1, upper_rank)]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

# The rest of your code remains the same
u_stat, p_value = stats.mannwhitneyu(just_nva_peak, rest_peak, alternative='two-sided')

result = hodges_lehmann_interval(just_nva_peak, rest_peak, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")


Mann Whitney U Statistics: 62.0
p value: 0.1044
Hodges-Lehmann Estimator: 91.61
Confidence Interval: (np.float64(90.76), np.float64(91.77000000000001))


In [106]:
#Peak movement 9/30 - 10/24

just_nva_peak = nva_comp_df.loc[nva_comp_df['ticker'] == 'NVA.AX', '9/30-10/24_peak'].values[0]
rest_peaks = nva_comp_df.loc[nva_comp_df['ticker'] != 'NVA.AX', '9/30-10/24_peak']

def hodges_lehmann_interval(just_nva_peak, rest_peaks, alpha=0.05):
    if np.isscalar(just_nva_peak):
        just_nva_peak = [just_nva_peak]
    if np.isscalar(rest_peaks):
        rest_peaks = [rest_peaks]

    pairwise_differences = [x1 - x2 for x1 in just_nva_peak for x2 in rest_peaks]
    pairwise_differences.sort()

    hl_estimator = np.median(pairwise_differences)

    n = len(pairwise_differences)

    _, u_stat = mannwhitneyu(just_nva_peak, rest_peaks, alternative='two-sided')

    lower_rank = int(max(0, u_stat - 1.96 * np.sqrt(n)))  # Ensure lower rank is >= 0
    upper_rank = int(min(n - 1, u_stat + 1.96 * np.sqrt(n)))  # Ensure upper rank < n

    lower_bound = pairwise_differences[lower_rank]
    upper_bound = pairwise_differences[upper_rank]

    return {
        "Hodges-Lehmann Estimator": hl_estimator,
        "Confidence Interval": (lower_bound, upper_bound)
    }

u_stat, p_value = stats.mannwhitneyu(just_nva_peak, rest_peaks, alternative = 'two-sided')


result = hodges_lehmann_interval(just_nva_peak, rest_peaks, alpha=0.05)
print(f'Mann Whitney U Statistics: {u_stat}')
print(f'p value: {p_value:.4f}')
print(f"Hodges-Lehmann Estimator: {result['Hodges-Lehmann Estimator']}")
print(f"Confidence Interval: {result['Confidence Interval']}")


Mann Whitney U Statistics: 62.0
p value: 0.1044
Hodges-Lehmann Estimator: 91.61
Confidence Interval: (np.float64(-69.55), np.float64(72.03))


In [113]:
#For combining peak movement p - values
from scipy.stats import chi2

def fisher_combined_pvalue(p_values):
 
    chi2_stat = -2 * np.sum(np.log(p_values))

    df = 2 * len(p_values)

    combined_p_value = 1 - chi2.cdf(chi2_stat, df)

    return combined_p_value

p_values = [0.4485, 0.1044]  # List of individual p-values
combined_p_value = fisher_combined_pvalue(p_values)

print(f"Combined p-value using Fisher's method: {combined_p_value:.4f}")


Combined p-value using Fisher's method: 0.1902
